In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, year, month
from pyspark.sql.types import DateType

In [2]:
# Inicializar a SparkSession
spark = SparkSession.builder \
    .appName("Global Temperature by date Analysis") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/09/08 20:20:51 WARN Utils: Your hostname, DESKTOP-AA2785H resolves to a loopback address: 127.0.1.1; using 172.18.127.117 instead (on interface eth0)
24/09/08 20:20:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 20:20:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Carregar o dataset
df = spark.read.load('abfss://temperaturebigdata@brenoragestorage1.dfs.core.windows.net/GlobalLandTemperaturesByCity.csv', format='csv', header=True)

In [4]:
df.printSchema()
df.show(5)

root
 |-- dt: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              NULL

Limpeza dos Dados

In [5]:
from pyspark.sql.functions import col, sum

# Criar uma lista de expressões para contar valores nulos por coluna
null_counts = [sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]

# Aplicar as expressões e mostrar o resultado
df.select(null_counts).show()

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
|  0|            364130|                       364130|   0|      0|       0|        0|
+---+------------------+-----------------------------+----+-------+--------+---------+



In [6]:
# Criar uma lista de expressões para contar valores nulos por coluna
null_counts = [sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]

In [7]:
# Aplicar as expressões e mostrar o resultado
df.select(null_counts).show()

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
|  0|            364130|                       364130|   0|      0|       0|        0|
+---+------------------+-----------------------------+----+-------+--------+---------+



In [8]:
from pyspark.sql.functions import col

In [9]:
# Remover linhas onde 'AverageTemperature''Country' 'City' e 'dt' são nulos
df_cleaned = df.dropna(subset=['AverageTemperature', 'Country', 'City', 'dt'])

In [10]:
# Preencher valores nulos em outras colunas com um valor padrão (opcional)
df_cleaned = df_cleaned.fillna({
    'AverageTemperatureUncertainty': 0,
    'City': 'Unknown',
    'Latitude': 'Unknown',
    'Longitude': 'Unknown'
})

In [11]:
# Verificar se a limpeza foi bem-sucedida
df_cleaned.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_cleaned.columns]).show()

+---+------------------+-----------------------------+----+-------+--------+---------+
| dt|AverageTemperature|AverageTemperatureUncertainty|City|Country|Latitude|Longitude|
+---+------------------+-----------------------------+----+-------+--------+---------+
|  0|                 0|                            0|   0|      0|       0|        0|
+---+------------------+-----------------------------+----+-------+--------+---------+



In [12]:
from pyspark.sql.functions import avg
from pyspark.sql.functions import avg, to_date, col

In [13]:
# Certificar-se de que a coluna 'dt' está no formato de data
df = df.withColumn("dt", to_date(col("dt")))

In [14]:
# Adicionar colunas de mês e ano
df = df.withColumn("month", month(col("dt")))
df = df.withColumn("year", year(col("dt")))
df.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+-----+----+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|month|year|
+----------+------------------+-----------------------------+-----+-------+--------+---------+-----+----+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|   11|1743|
|1743-12-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|   12|1743|
|1744-01-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|    1|1744|
|1744-02-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|    2|1744|
|1744-03-01|              NULL|                         NULL|Århus|Denmark|  57.05N|   10.33E|    3|1744|
+----------+------------------+-----------------------------+-----+-------+--------+---------+-----+----+
only showing top 5 rows



In [15]:
# Calcular a temperatura média por país, cidade e data
df_country_city_avg_temp = df_cleaned.groupBy("Country", "City", "dt").agg(
    avg("AverageTemperature").alias("AverageTemperature")
)

In [16]:
# Coalesce para um único partição
df_cleaned_coalesced = df_cleaned.coalesce(1)

In [17]:
# Salvar o dataset limpo
df_cleaned_coalesced.write.csv("abfss://temperaturebigdata@brenoragestorage1.dfs.core.windows.net/cleanedUnico_bytemperature_bydatabycity.csv", header=True, mode="overwrite")

In [18]:
# Interromper Spark
spark.stop()